In [89]:
from dotenv import dotenv_values
from openai import OpenAI
import time

In [6]:
OPENAI_API_KEY = dotenv_values(".env")["OPENAI_API_KEY"]
client = OpenAI(api_key=OPENAI_API_KEY)

In [10]:
GPT_MODEL = "gpt-3.5-turbo-1106"

# Overview
input: prompt of what the website should be.

output: code + ai execution of the website. 
1. Create webpage
   1. Executive detailing necessary components
   2. For each component: Provide previously solved context (summary of what's been done)
      1. Assistant asking questions
      2. Executive answering questions
      3. REPEAT for some steps
   3. Combine components
   4. Host
2. Test webpage
   1. Test connectivity using agent + code interpreter
   2. Test UI locally, and with semantic interpretation of the code compared to initial prompt


# Notes
Context window is likely large enough to keep everything in there without summary. 

In [90]:
# ... i need to shorten this
def communicative_dehallucination(
    thread_id, executive, developer, previous_components, component, component_specification, max_iter=2
):
    """
    Implements a communicative dehallucination process for software development.

    This function facilitates an iterative Q&A process between a developer and an executive
    to clarify component specifications and reduce potential hallucinations in AI-generated code.
    It then generates an implementation based on the clarified context and provides a summary
    of what has been achieved.

    Args:
        thread_id (str): The ID of the conversation thread.
        executive (object): The executive assistant object for answering questions.
        developer (object): The developer assistant object for asking questions and implementing.
        previous_components (str): A summary of previously implemented components.
        component (str): The current component to be implemented.
        component_specification (str): Initial specifications for the current component.
        max_iter (int, optional): Maximum number of Q&A iterations. Defaults to 3.

    Returns:
        tuple: A tuple containing:
            - implementation (str): The generated implementation of the component.
            - context (str): The full context including all Q&A pairs.
            - summary (str): A concise summary of what has been achieved for this component.

    Raises:
        Any exceptions raised by the OpenAI API calls are not caught in this function.

    Note:
        This function assumes the existence of a configured OpenAI client named 'client'.
    """

    print(f"\n--- Starting communicative dehallucination for component: {component} ---")
    print(f"Previous components: {previous_components}")
    print(f"Component specification: {component_specification}")
    
    # Initialize the context with previous information
    context = f"Previous Components: {previous_components}\nCurrent Component: {component}\nSpecification: {component_specification}"
    
    # Iterative Q&A process
    for i in range(max_iter):
        print(f"\n--- Iteration {i+1}/{max_iter} ---")
        
        # Developer asks a question
        message = client.beta.threads.messages.create(
            thread_id=thread_id,
            role="user",
            content=f"Context: {context}\n Component Specification: {component}. As the developer, ask a clarifying question about this component",
        )
        
        run = client.beta.threads.runs.create(
            thread_id=thread_id,
            assistant_id=developer.id,
            instructions="Ask a specific question to clarify details about the component",
        )
        
        while run.status != "completed":
            time.sleep(1)
            run = client.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run.id)
        
        # Retrieve the developer's question
        messages = client.beta.threads.messages.list(thread_id=thread_id)
        question = messages.data[0].content[0].text.value
        print(f"Developer's question: {question}")
        
        # Executive answers the question
        message = client.beta.threads.messages.create(
            thread_id=thread_id,
            role="user",
            content=f"Developer's Question: {question}\n Context: {context}\n Component Specification: {component}. As the senior advisor, answer with specificity the developer's question about this component."
        )
        
        # Retrieve the executive's answer
        messages = client.beta.threads.messages.list(thread_id=thread_id)
        answer = messages.data[0].content[0].text.value
        print(f"Executive's answer: {answer}")
        
        # Update context with new Q&A pair
        context += f"\nQ: {question}\nA: {answer}"
        
    print("\n--- Implementing component ---")
    # Developer implements component based on clarified context
    message = client.beta.threads.messages.create(
        thread_id=thread_id,
        role="user",
        content=f"Based on this context, implement the component: {context}",
    )
    
    run = client.beta.threads.runs.create(
        thread_id=thread_id,
        assistant_id=developer.id,
        instructions="Implement the component based on the provided context and specifications.",
    )
    
    while run.status != "completed":
            time.sleep(1)
            run = client.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run.id)
            
    # Retrieve the implementation
    messages = client.beta.threads.messages.list(thread_id=thread_id)
    implementation = messages.data[0].content[0].text.value
    print(f"Implementation:\n{implementation}")
    
    print("\n--- Generating summary ---")
    # Generate a summary of what has been achieved
    message = client.beta.threads.messages.create(
        thread_id=thread_id,
        role="user",
        content=f"Summarize what has been achieved in implementing this component: {implementation}",
    )
    
    run = client.beta.threads.runs.create(
        thread_id=thread_id,
        assistant_id=executive.id,
        instructions="Provide a concise summary of what has been achieved in implementing this component. This summary will be used as context for the next component.",
    )
    
    while run.status != "completed":
        time.sleep(1)
        run = client.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run.id)
        
    # Retrieve the summary
    messages = client.beta.threads.messages.list(thread_id=thread_id)
    summary = messages.data[0].content[0].text.value
    print(f"Summary: {summary}")
    
    print("\n--- Communicative dehallucination complete ---")
    return implementation, context, summary

In [86]:
executive = client.beta.assistants.create(
    name="Executive",
    instructions="You are a senior software developer. You break down tasks and provide high-level instructions using natural language.",
    model=GPT_MODEL,
)

developer = client.beta.assistants.create(
    name="Developer",
    instructions="You are a web developer. You create and deploy Flask webpages based on instructions, asking questions when necessary",
    model=GPT_MODEL,
)

In [80]:
WEBSITE_IDEA = "Provide the code to quickstart a basic builtin Flask server."
thread = client.beta.threads.create()

initial_prompt = client.beta.threads.messages.create(
    thread_id=thread.id, role="user", content=WEBSITE_IDEA
)

In [81]:
run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id,
    assistant_id=executive.id,
    # List, in natural language, only the essential code components needed to fulfill the user's request.
    instructions="""
    List, in natural language, only the essential code components needed to fulfill the user's request.
   
    Your response must:
    1. Include only core components.
    2. Be numbered with each component on a new line.
    3. Focus on the conceptual steps of specific code elements or function calls
    4. Be comprehensive, covering all necessary components
    5. Use technical terms appropriate for the specific programming language and framework.
   
    Important:
    - Assume all dependencies are already installed but not imported.
    - Do not include dependency installations.
    - Focus solely on the code components needed to implement the functionality.
   
    Example format:
    1. [Natural language specification of the specific code component or function call]
    2. [Natural language specification of the specific code component or function call]
    ...
    """,
)


In [82]:
if run.status == "completed":
    messages = client.beta.threads.messages.list(thread_id=thread.id)
    for message in messages:
        print(message.content[0].text.value, '\n\n')
        
    messages = client.beta.threads.messages.list(thread_id=thread.id)
    
    message = client.beta.threads.messages.retrieve(
        thread_id=thread.id, message_id=messages.data[0].id
    )

    print(message.content[0].text.value)
else:
    print(run.status)

1. Import the Flask class from the flask module.
2. Create an instance of the Flask class.
3. Define a route using the route decorator.
4. Create a function to handle the defined route.
5. Use the run method to start the Flask application. 


Provide the code to quickstart a basic builtin Flask server. 


1. Import the Flask class from the flask module.
2. Create an instance of the Flask class.
3. Define a route using the route decorator.
4. Create a function to handle the defined route.
5. Use the run method to start the Flask application.


In [87]:
# Parse the components from the executive's response
components = message.content[0].text.value.split('\n')
components = [comp.strip() for comp in components if comp.strip()]

# Initialize previous_components
previous_components = ""

In [91]:
for i, component in enumerate(components):
    print(f"\n=== Processing Component {i+1}: {component} ===")
    
    # Extract the component specification (removing the number at the beginning)
    component_spec = component.split('. ', 1)[1] if '. ' in component else component
    
    # Run communicative_dehallucination for each component
    implementation, context, summary = communicative_dehallucination(
        thread_id=thread.id,
        executive=executive,
        developer=developer,
        previous_components=previous_components,
        component=component_spec,
        component_specification=component_spec,
        max_iter=2
    )
    
    # Update previous_components for the next iteration
    previous_components += f"\n{summary}"


=== Processing Component 1: 1. Import the Flask class from the flask module. ===

--- Starting communicative dehallucination for component: Import the Flask class from the flask module. ---
Previous components: 
Component specification: Import the Flask class from the flask module.

--- Iteration 1/2 ---
Developer's question: Do you have a specific requirement for the version of Flask or any particular import statement that needs clarification?
Executive's answer: Developer's Question: Do you have a specific requirement for the version of Flask or any particular import statement that needs clarification?
 Context: Previous Components: 
Current Component: Import the Flask class from the flask module.
Specification: Import the Flask class from the flask module.
 Component Specification: Import the Flask class from the flask module.. As the senior advisor, answer with specificity the developer's question about this component.

--- Iteration 2/2 ---
Developer's question: My apologies for 